# 5. Загрузите данные в ClickHouse и создайте таблицу для их хранения.

В нашём случае был запузен контейнер в Docker Desktop
![Docker_ClickHouse](screen/Docker_ClikHouse.png)

In [4]:
from clickhouse_driver import Client
import json

In [7]:
# Подключение к серверу ClickHouse
client = Client(host='localhost', user='default', password='', port=9000)

# Попытка выполнить запрос
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Произошла ошибка: {e}")

# Создание базы данных
client.execute('CREATE DATABASE IF NOT EXISTS W3T2_db')
client.execute('''
CREATE TABLE IF NOT EXISTS W3T2_db.storage_books (
    Title String,
    Price Float64,
    Stock_int String,
    Description String
) ENGINE = MergeTree()
ORDER BY Title
''')

# Загрузка данных из файла
with open('data_site_books.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


# Вставка данных в таблицу
for insert in data:
    client.execute("""
    INSERT INTO W3T2_db.storage_books (
        Title, Price, Stock_int, Description
        ) VALUES""",
        [(
            insert['Title'],
            insert['Price'],
            insert['Stock_int'],
            insert['Description'])])
    

# Вывод одной строки базы данных записаной в ClickHouse
result = client.execute("SELECT * FROM W3T2_db.storage_books LIMIT 1")
print("Первая запись таблицы:", result[0])

# Вывод всей таблицы из базы данных ClickHouse
result_all_books = client.execute("SELECT * FROM W3T2_db.storage_books")
for row in result_all_books:
    print(row)

[]
Вставленная запись: ('#HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.', 23.11, 'In stock (17 available)', "There is a cosmic alarm clock going off around the world! #HigherSelfie's aim is to unite all those waking up spiritually in this digital age. This book is a guide to love, connection, and kickass surrendered action for young people who have at least a toe in the door of spirituality. With a no-nonsense approach and full of wit and humor, this book shares age-old concepts There is a cosmic alarm clock going off around the world! #HigherSelfie's aim is to unite all those waking up spiritually in this digital age. This book is a guide to love, connection, and kickass surrendered action for young people who have at least a toe in the door of spirituality. With a no-nonsense approach and full of wit and humor, this book shares age-old concepts in a language that is accessible to the modern spiritual audience. Whether you have just bought a yoga mat or have been m

In [8]:
# Запрос количества и названий книг, дороже 50 фунтов
result_expensive_books = client.execute("SELECT COUNT(Title), Title FROM W3T2_db.storage_books WHERE Price > 50 GROUP BY Title")
print("Количество и названия книг, дороже 50 фунтов:", result_expensive_books)

Количество и названия книг, дороже 50 фунтов: [(3, 'The Black Maria'), (3, 'Soumission'), (3, 'A Light in the Attic'), (3, 'The Past Never Ends'), (3, 'Behind Closed Doors'), (3, 'Tipping the Velvet'), (3, 'The Pioneer Woman Cooks: Dinnertime: Comfort Classics, Freezer Food, 16-Minute Meals, and Other Delicious Ways to Solve Supper!'), (3, 'The Secret of Dreadwillow Carse'), (3, 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991'), (3, 'Birdsong: A Story in Pictures'), (3, "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)"), (3, 'Libertarianism for Beginners'), (3, 'Aladdin and His Wonderful Lamp'), (3, 'We Love You, Charlie Freeman'), (3, 'Sapiens: A Brief History of Humankind'), (3, 'Slow States of Collapse: Poems')]


In [9]:
# Запрос для определения максимальной и минимальной цены книги в коллекции
result_max_min_price = client.execute("SELECT MAX(Price), MIN(Price) FROM W3T2_db.storage_books")
print("Максимальная и минимальная цена книги в коллекции:", result_max_min_price)

Максимальная и минимальная цена книги в коллекции: [(57.31, 12.84)]
